## Imports

In [14]:
import numpy as np
import torch
import os
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

## Load the data

In [12]:
# Load the data from the specified directory
X_train = np.load('./ProcessedInputData/X_train.npy')
Y_train = np.load('./ProcessedInputData/Y_train.npy')
X_val = np.load('./ProcessedInputData/X_val.npy')
Y_val = np.load('./ProcessedInputData/Y_val.npy')

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
Y_val_tensor = torch.tensor(Y_val, dtype=torch.long)

# Create a DataLoader for batching
batch_size = 64
train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = TensorDataset(X_val_tensor, Y_val_tensor)
val_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

In [13]:
# Print the first few elements of the tensors
print("\nFirst few elements of Y_train_tensor:")
print(Y_train_tensor.size())
print(Y_train_tensor[:10])  # Print first 10 elements

print("First few elements of X_train_tensor:")
print(X_train_tensor.size())
# print(X_train_tensor[:3])  # Print first 3 elements

# each input is 5x90x1 matrix
# 5 bc there are 5 eeg channels recordings per sample,
# 90 bc i did fourier transform and binned it into 90 different bins from 0.5 to 40 Hz (relevant frequencies for EEG)
# the 1 was added just to be the dimension of picture style for CNN so its like black and white

print("Val tensors:")
print(Y_val_tensor.size())
print(X_val_tensor.size())


First few elements of Y_train_tensor:
torch.Size([3153])
tensor([1, 1, 1, 1, 0, 0, 0, 1, 0, 0])
First few elements of X_train_tensor:
torch.Size([3153, 5, 90, 1])
Val tensors:
torch.Size([394])
torch.Size([394, 5, 90, 1])


## Simple Test Net

This is a simple test net which uses a 5 element sanity dataset and attempts to overfit to this dataset. As is it does not appear to work but at least some of the sizes in the inputs / outputs appear to be correct

In [46]:
sanity_dataset = TensorDataset(X_train_tensor[:20], Y_train_tensor[:20]) # take just a few elements
sanity_loader = DataLoader(sanity_dataset, batch_size=5, shuffle=True)

# Define a simple neural network for binary classification
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(SimpleNN, self).__init__()
        self.hidden = nn.Linear(input_size, hidden_size)  # Hidden layer
        self.relu = nn.ReLU()  # ReLU activation function
        self.output = nn.Linear(hidden_size, 1)  # Output layer (single neuron)

    def forward(self, x):
        x = x.view(-1, 5*90*1)  # Flatten the tensor
        x = self.relu(self.hidden(x))  # Apply hidden layer and activation
        x = self.output(x)  # Apply output layer
        return x  # Output is raw logits (before sigmoid)

# Function to train the model
def train_model(model, train_loader, epochs=10):
    # Fixed PyTorch random seed for reproducible result
    torch.manual_seed(1000)
    model.train()  # Set model to training mode
    criterion = nn.BCEWithLogitsLoss()  # Binary Cross-Entropy with Logits
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        
        for inputs, labels in train_loader:
            
            # Zero the gradients
            optimizer.zero_grad()
            # Forward pass
            outputs = model(inputs)

            print("Input:", inputs.size())
            print("Output:", outputs.size(), outputs)
            print("Label:", labels.size(), labels)
            labels = labels.view(-1, 1).float()  # reshape labels to [batch_size, 1]
            print("Label Reshaped:", labels.size(), labels)

            # Calculate loss
            loss = criterion(outputs, labels)  # Convert labels to float for BCE
            loss.backward()
            # Update weights
            optimizer.step()
            # Track loss and accuracy
            running_loss += loss.item()
            # Convert logits to probabilities using sigmoid
            predicted = torch.round(torch.sigmoid(outputs))  # Round the output to 0 or 1

            print("Predicted:", predicted.size(), predicted)
            
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        # Calculate the average loss and accuracy for the epoch
        avg_loss = running_loss / len(train_loader)
        accuracy = 100 * correct / total

        # Print results
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%')

# Instantiate the model, loss function, and optimizer
sanity_NN = SimpleNN(input_size = 90*5*1, hidden_size = 64)

# Call the train_model function
train_model(model = sanity_NN, train_loader = sanity_loader, epochs = 10)


Input: torch.Size([5, 5, 90, 1])
Output: torch.Size([5, 1]) tensor([[ 5.8607e+01],
        [ 9.8089e+01],
        [ 6.1685e+01],
        [ 9.2391e+00],
        [-1.0584e-02]], grad_fn=<AddmmBackward0>)
Label: torch.Size([5]) tensor([0, 1, 1, 1, 1])
Label Reshaped: torch.Size([5, 1]) tensor([[0.],
        [1.],
        [1.],
        [1.],
        [1.]])
Predicted: torch.Size([5, 1]) tensor([[1.],
        [1.],
        [1.],
        [1.],
        [0.]], grad_fn=<RoundBackward0>)
Input: torch.Size([5, 5, 90, 1])
Output: torch.Size([5, 1]) tensor([[-214822.0000],
        [ -26762.9414],
        [ -14989.7725],
        [  -4071.5464],
        [ -48059.1328]], grad_fn=<AddmmBackward0>)
Label: torch.Size([5]) tensor([1, 0, 1, 1, 0])
Label Reshaped: torch.Size([5, 1]) tensor([[1.],
        [0.],
        [1.],
        [1.],
        [0.]])
Predicted: torch.Size([5, 1]) tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.]], grad_fn=<RoundBackward0>)
Input: torch.Size([5, 5, 90, 1]